In [8]:
import fiftyone as fo
import fiftyone.zoo as foz

classes_to_keep = [
    "person",
    "bicycle",
    "car",
    "motorcycle",
    "airplane",
]

train_dataset_name = "coco_train_1000"
if train_dataset_name in fo.list_datasets():
    # Delete the existing dataset
    fo.Dataset(train_dataset_name).delete()

val_dataset_name = "coco_val_100"
if val_dataset_name in fo.list_datasets():
    # Delete the existing dataset
    fo.Dataset(val_dataset_name).delete()

train_dataset = foz.load_zoo_dataset(
    "coco-2017",
    label_types=["detections"],
    split="train",
    classes=classes_to_keep,
    max_samples=1000,
    shuffle=True,
    seed=123,
    only_matching=True,
    num_workers=10,
    dataset_name=train_dataset_name,
)

val_dataset = foz.load_zoo_dataset(
    "coco-2017",
    label_types=["detections"],
    split="validation",
    classes=classes_to_keep,
    max_samples=100,
    shuffle=True,
    seed=123,
    only_matching=True,
    num_workers=10,
    dataset_name=val_dataset_name,
)


Found annotations at '/home/yipeng/fiftyone/coco-2017/raw/instances_train2017.json'
Sufficient images already downloaded
Existing download of split 'train' is sufficient
Loading 'coco-2017' split 'train'
 100% |███████████████| 1000/1000 [1.9s elapsed, 0s remaining, 525.9 samples/s]      
Dataset 'coco_train_1000' created
Found annotations at '/home/yipeng/fiftyone/coco-2017/raw/instances_val2017.json'
Sufficient images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |█████████████████| 100/100 [212.9ms elapsed, 0s remaining, 469.6 samples/s]     
Dataset 'coco_val_100' created


In [9]:
import os
import cv2 as cv

output_dir = "resized_dataset"
os.makedirs(output_dir, exist_ok=True)

resized_image_size = (256, 256)

def resize_image_and_annotations(sample):

    # Resize image
    image_path = sample.filepath
    image = cv.imread(image_path)
    height, width = image.shape[:2]
    image = cv.resize(image, resized_image_size)

    # Update the file path
    resized_image_path = os.path.join(output_dir, os.path.basename(image_path))
    sample.filepath = resized_image_path

    # Save the resized image
    cv.imwrite(resized_image_path, image)

    # Update the bounding boxes
    scale_x = resized_image_size[0] / float(width)
    scale_y = resized_image_size[1] / float(height)

    for det in sample["ground_truth"].detections:
        x1, y1, x2, y2 = det.bounding_box
        x1 = x1 * scale_x
        x2 = x2 * scale_x
        y1 = y1 * scale_y
        y2 = y2 * scale_y
        det.bounding_box = [x1, y1, x2, y2]

    sample.save()

for sample in train_dataset:
    resize_image_and_annotations(sample)

for sample in val_dataset:
    resize_image_and_annotations(sample)

